In [162]:
%load_ext autoreload
%autoreload 2

In [1]:
from From_Shelter_to_Love.data_provisoria import get_data

In [2]:
df = get_data()

In [3]:
df = df.drop(columns = ["Animal ID", "age_upon_intake_years", "age_upon_outcome_years"] , axis=1)
df = df[df['days_in_shelter'] < 60]

In [4]:
df

,Intake Type,Intake Condition,Animal Type,Breed,Outcome Type,days_in_shelter,age_upon_intake_months,age_upon_outcome_months,neutered_or_spayed_outcome,male_or_female_outcome,neutered_or_spayed_intake,male_or_female_intake,group_color
0,Public Assist,Normal,Dog,Mixed,Return to Owner,2.0,72.0,72.0,1,1.0,1,1.0,Yellow
1,Owner Surrender,Normal,Dog,Dachshund,Transfer,5.0,120.0,120.0,1,1.0,1,1.0,Tricolor
2,Public Assist,Injured,Dog,Shetland Sheepdog,Return to Owner,1.0,192.0,192.0,1,1.0,1,1.0,Brown
3,Stray,Aged,Dog,Mixed,Return to Owner,1.0,180.0,180.0,1,0.0,1,0.0,Black
4,Stray,Normal,Dog,Mixed,Return to Owner,1.0,180.0,180.0,0,0.0,0,0.0,Black
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110852,Stray,Normal,Dog,Mixed,Return to Owner,2.0,24.0,24.0,0,1.0,0,1.0,Black
110853,Stray,Normal,Dog,Mixed,Return to Owner,2.0,24.0,24.0,1,0.0,1,0.0,White
110861,Stray,Normal,Dog,Mixed,Return to Owner,1.0,24.0,24.0,1,1.0,1,1.0,Brown
110874,Owner Surrender,Normal,Dog,Alaskan Husky,Rto-Adopt,2.0,12.0,12.0,1,1.0,0,1.0,Black


In [150]:
df.shape

(94178, 11)

In [151]:
df.dtypes

Intake Type                 object
Intake Condition            object
Animal Type                 object
Breed                       object
Outcome Type                object
days_in_shelter            float64
age_upon_intake_months     float64
age_upon_outcome_months    float64
neutered_or_spayed           int64
male_or_female             float64
group_color                 object
dtype: object

In [153]:
from sklearn.model_selection import train_test_split

y = df["days_in_shelter"]
X = df.drop(columns = ["days_in_shelter"] , axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 10)

In [154]:
y_train.shape

(65924,)

In [155]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

In [156]:
from sklearn import set_config ;set_config(display='diagram')
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler , FunctionTransformer,OneHotEncoder
from sklearn.compose import make_column_selector,ColumnTransformer
from sklearn.linear_model import LinearRegression

# Impute then Scale for numerical variables
num_transformer = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', MinMaxScaler())])

# Encode categorical varibles 
cat_transformer = OneHotEncoder(handle_unknown='ignore',sparse=False)

# Apply transformations to desired features
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, make_column_selector(dtype_include=['int64',"float64"])),
    ('cat_transformer', cat_transformer, make_column_selector(dtype_include=["object"]))])

final_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('linear_regression', LinearRegression())])

final_pipe_SGD = Pipeline([
    ('preprocessing', preprocessor),
    ('linear_regression', SGDRegressor())])

In [157]:
final_pipe_trained = final_pipe.fit(X_train,y_train)
final_pipe_trained.score(X_test,y_test)

-4.0376116067402644e+18

In [158]:
final_pipe_SGD_trained = final_pipe_SGD.fit(X_train,y_train)
final_pipe_SGD_trained.score(X_test,y_test)

0.22895906964070933

In [159]:
from sklearn.model_selection import cross_validate, cross_val_score

# Cross validate pipeline
cross_val_score(final_pipe, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()

-3.2239896142778117e+18

In [160]:
from sklearn.model_selection import cross_validate, cross_val_score

# Cross validate pipeline
cross_val_score(final_pipe_SGD, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()

-130.3827428999406

In [161]:
cv_results = cross_validate(final_pipe, X_train, y_train, cv=5)
cv_results

{'fit_time': array([1.72568941, 1.51526141, 2.17139935, 4.66699266, 3.15224075]),
 'score_time': array([0.08465219, 0.06908703, 0.09852266, 0.23815632, 0.15608644]),
 'test_score': array([-8.92307655e+15, -4.77844663e+16, -8.89984731e+11, -3.38938622e+15,
        -3.68829193e+16])}

In [130]:
#cv_results = cross_validate(model, X_train, y_train, cv=5, scoring=['max_error', 'r2', 'neg_mean_absolute_error', 'neg_mean_squared_error'])
